In [20]:
import numpy as np
from collections import defaultdict


def simulate_women(Q, num_women=1000, obs_interval=48):
    n_states = Q.shape[0]
    absorbing_state = n_states - 1
    time_series = []
    
    for _ in range(num_women):
        current_state = 0
        time = 0
        woman_series = [current_state + 1]
        
        while current_state != absorbing_state:
            rate = -Q[current_state, current_state]
            holding_time = np.random.exponential(1/rate) if rate > 0 else float('inf')
            
            if current_state != absorbing_state:
                possible_states = [i for i in range(n_states) if i != current_state]
                probs = Q[current_state, possible_states] / rate
                next_state = np.random.choice(possible_states, p=probs)
            else:
                next_state = current_state
            
            next_obs_time = (time // obs_interval + 1) * obs_interval
            transition_time = time + holding_time
            
            while next_obs_time < transition_time:
                woman_series.append(current_state + 1)
                next_obs_time += obs_interval
            
            time = transition_time
            current_state = next_state
        
        if woman_series[-1] != absorbing_state + 1:
            woman_series.append(absorbing_state + 1)
        
        time_series.append(woman_series)
    
    return time_series

def estimate_Q_from_partial_obs(time_series, n_states=5, obs_interval=48):
    Nij = np.zeros((n_states, n_states))
    Si = np.zeros(n_states)
    
    for series in time_series:
        prev_time = 0
        for i in range(1, len(series)):
            current_time = i * obs_interval
            prev_state = series[i-1] - 1 
            current_state = series[i] - 1
            
            Si[prev_state] += obs_interval
            
            if prev_state != current_state:
                Nij[prev_state, current_state] += 1
    
    Q_est = np.zeros((n_states, n_states))
    for i in range(n_states):
        if Si[i] > 0:
            for j in range(n_states):
                if i != j:
                    Q_est[i, j] = Nij[i, j] / Si[i]

    for i in range(n_states):
        Q_est[i, i] = -np.sum(Q_est[i, :])
    
    return Q_est


Q = np.array([
    [-0.0085, 0.005, 0.0025, 0.0, 0.001],
    [0.0, -0.014, 0.005, 0.004, 0.005],
    [0.0, 0.0, -0.008, 0.003, 0.005],
    [0.0, 0.0, 0.0, -0.009, 0.009],
    [0.0, 0.0, 0.0, 0.0, 0.0]
])

# Simulate 1000 women
time_series = simulate_women(Q, num_women=1000, obs_interval=48)

# Estimate Q from partial observations
Q_estimated = estimate_Q_from_partial_obs(time_series, n_states=5, obs_interval=48)

print("Original Q matrix:")
print(Q)
print("\nEstimated Q matrix:")
print(Q_estimated)

# Compare original and estimated matrices
print("\nComparison (Original vs Estimated):")
for i in range(5):
    print(f"State {i+1}:")
    print(f"Original: {Q[i,:]}")
    print(f"Estimated: {Q_estimated[i,:]}")
    print()

Original Q matrix:
[[-0.0085  0.005   0.0025  0.      0.001 ]
 [ 0.     -0.014   0.005   0.004   0.005 ]
 [ 0.      0.     -0.008   0.003   0.005 ]
 [ 0.      0.      0.     -0.009   0.009 ]
 [ 0.      0.      0.      0.      0.    ]]

Estimated Q matrix:
[[-0.00702168  0.00286485  0.00219077  0.00052663  0.00143945]
 [ 0.         -0.01025332  0.00314133  0.00261359  0.00449839]
 [ 0.          0.         -0.0066116   0.00201222  0.00459937]
 [ 0.          0.          0.         -0.00731159  0.00731159]
 [ 0.          0.          0.          0.         -0.        ]]

Comparison (Original vs Estimated):
State 1:
Original: [-0.0085  0.005   0.0025  0.      0.001 ]
Estimated: [-0.00702168  0.00286485  0.00219077  0.00052663  0.00143945]

State 2:
Original: [ 0.    -0.014  0.005  0.004  0.005]
Estimated: [ 0.         -0.01025332  0.00314133  0.00261359  0.00449839]

State 3:
Original: [ 0.     0.    -0.008  0.003  0.005]
Estimated: [ 0.          0.         -0.0066116   0.00201222  0.0045993

In [ ]:
import numpy as np
from collections import defaultdict

def simulate_between_observations(Q, observed_series, obs_interval=48, n_simulations=100):
    n_states = Q.shape[0]
    all_transitions = []
    all_sojourns = []
    
    for series in observed_series:
        transitions = defaultdict(int)
        sojourns = defaultdict(float)
        
        for i in range(len(series)-1):
            start_state = series[i] - 1
            end_state = series[i+1] - 1
            time_passed = 0
            current_state = start_state
            
            while time_passed < obs_interval:
                rate = -Q[current_state, current_state]
                holding_time = np.random.exponential(1/rate) if rate > 0 else float('inf')
                
                if current_state != n_states - 1:
                    possible_states = [i for i in range(n_states) if i != current_state]
                    probs = Q[current_state, possible_states] / rate
                    next_state = np.random.choice(possible_states, p=probs)
                else:
                    next_state = current_state
                
                time_remaining = obs_interval - time_passed
                time_in_state = min(holding_time, time_remaining)
                
                sojourns[current_state] += time_in_state
                
                if time_passed + holding_time <= obs_interval:
                    transitions[(current_state, next_state)] += 1
                    current_state = next_state
                else:
                    if current_state == end_state:
                        break
                    else:
                        transitions = defaultdict(int)
                        sojourns = defaultdict(float)
                        time_passed = 0
                        current_state = start_state
                        continue
                
                time_passed += holding_time
        
        all_transitions.append(transitions)
        all_sojourns.append(sojourns)
    
    return all_transitions, all_sojourns

def estimate_Q_from_simulations(all_transitions, all_sojourns, n_states=5):
    """Estimate Q matrix from simulated transitions and sojourn times"""
    Nij = np.zeros((n_states, n_states))
    Si = np.zeros(n_states)
    
    for transitions, sojourns in zip(all_transitions, all_sojourns):
        for (i, j), count in transitions.items():
            Nij[i, j] += count
        for state, time in sojourns.items():
            Si[state] += time
    
    Q_est = np.zeros((n_states, n_states))
    for i in range(n_states):
        if Si[i] > 0:
            for j in range(n_states):
                if i != j:
                    Q_est[i, j] = Nij[i, j] / Si[i]
    
    for i in range(n_states):
        Q_est[i, i] = -np.sum(Q_est[i, :])
    
    return Q_est

def mc_em_algorithm(Q_init, observed_series, obs_interval=48, 
                   max_iter=100, tolerance=1e-3, n_simulations=100):
    Q_current = Q_init.copy()
    prev_Q = Q_current.copy()
    
    for iteration in range(max_iter):
        # Simulate paths
        all_transitions, all_sojourns = simulate_between_observations(
            Q_current, observed_series, obs_interval, n_simulations)
        
        # Estimate new Q matrix
        Q_current = estimate_Q_from_simulations(all_transitions, all_sojourns)
        
        # Check convergence
        max_diff = np.max(np.abs(Q_current - prev_Q))
        print(f"Iteration {iteration+1}: Max difference = {max_diff:.6f}")
        
        if max_diff < tolerance:
            print(f"Converged after {iteration+1} iterations")
            break
            
        prev_Q = Q_current.copy()
    
    return Q_current

# Define the given Q matrix (0-indexed)
Q_true = np.array([
    [-0.0085, 0.005, 0.0025, 0.0, 0.001],
    [0.0, -0.014, 0.005, 0.004, 0.005],
    [0.0, 0.0, -0.008, 0.003, 0.005],
    [0.0, 0.0, 0.0, -0.009, 0.009],
    [0.0, 0.0, 0.0, 0.0, 0.0]
])

# Simulate observed data (1000 women)
print("Simulating observed data...")
observed_series = simulate_women(Q_true, num_women=1000, obs_interval=48)

# Initialize Q with random values (or simple estimates)
Q_init = np.array([
    [-0.01, 0.004, 0.003, 0.0, 0.003],
    [0.0, -0.01, 0.005, 0.003, 0.002],
    [0.0, 0.0, -0.01, 0.004, 0.006],
    [0.0, 0.0, 0.0, -0.01, 0.01],
    [0.0, 0.0, 0.0, 0.0, 0.0]
])

Q_estimated = mc_em_algorithm(Q_init, observed_series, obs_interval=48, 
                            tolerance=1e-3, n_simulations=100)

print("\nTrue Q matrix:")
print(Q_true)
print("\nEstimated Q matrix:")
print(Q_estimated)

# Calculate relative errors
relative_error = np.abs((Q_estimated - Q_true) / Q_true)
relative_error[np.isinf(relative_error)] = 0  # Handle division by zero
print("\nRelative errors:")
print(relative_error)

Simulating observed data...
Iteration 1: Max difference = 0.033752
Iteration 2: Max difference = 0.013658
Iteration 3: Max difference = 0.002972
Iteration 4: Max difference = 0.002772
Iteration 5: Max difference = 0.003079
Iteration 6: Max difference = 0.001575
Iteration 7: Max difference = 0.002195
Iteration 8: Max difference = 0.002125
Iteration 9: Max difference = 0.001957
Iteration 10: Max difference = 0.001262
Iteration 11: Max difference = 0.001387
Iteration 12: Max difference = 0.001608
Iteration 13: Max difference = 0.003259
Iteration 14: Max difference = 0.005028
Iteration 15: Max difference = 0.002620
Iteration 16: Max difference = 0.006605
Iteration 17: Max difference = 0.002117
Iteration 18: Max difference = 0.003179
Iteration 19: Max difference = 0.001801
Iteration 20: Max difference = 0.001420
Iteration 21: Max difference = 0.002058
Iteration 22: Max difference = 0.004210
Iteration 23: Max difference = 0.001675
Iteration 24: Max difference = 0.002148
Iteration 25: Max dif

/var/folders/_d/_g85y1w11n34_6vgjpchq3_m0000gn/T/ipykernel_43122/1646394437.py:136: RuntimeWarning: invalid value encountered in divide
  relative_error = np.abs((Q_estimated - Q_true) / Q_true)
